In [80]:
import sys
import os
sys.path.insert(0, os.path.abspath('../py'))
from Secrets import ReadSecrets

from pandas_datareader import data as pdr

import pandas as pd

import yfinance as yf
yf.pdr_override()
from datetime import datetime as calendar
import numpy as np
import chart_studio.plotly as plty

secrets = ReadSecrets().read('../secrets.json')
plty.sign_in(secrets.secrets['plotly']['username'], secrets.secrets['plotly']['apiKey'])

import plotly.graph_objs as gobjs

In [81]:
def get_history(tickers, start_date, end_date):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=start_date, end=end_date))
        
    tickers_data = map(data, tickers)
    
    return pd.concat(tickers_data, keys=tickers, names = ['Ticker', 'Date'])

In [82]:
tickers = ['AAPL', 'MSFT', '^GSPC']
start_date = calendar(2016, 1 ,1)
end_date = calendar(2020, 2, 1)

data = get_history(tickers, start_date, end_date)
data = data.reset_index()
data = data.set_index(['Date', 'Ticker']).sort_index()
close = data['Close']

weekdays = pd.date_range(start=start_date, end=end_date, freq='B') # all weekdays in the interval
close = close.reindex(pd.MultiIndex.from_product([weekdays, tickers], names=['Date', 'Ticker']), fill_value=np.NaN)

close = close.reset_index().pivot(index='Date', columns='Ticker', values='Close')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [83]:
data_pct = data.pct_change(1)
display(data_pct)

Open       High        Low      Close  Adj Close  \
Date       Ticker                                                          
2016-01-04 AAPL          NaN        NaN        NaN        NaN        NaN   
           MSFT    -0.470617  -0.479928  -0.476569  -0.479829  -0.485443   
           ^GSPC   36.522091  36.193430  36.266905  35.727373  38.825873   
2016-01-05 AAPL    -0.948116  -0.948067  -0.948529  -0.948968  -0.952425   
           MSFT    -0.480567  -0.476712  -0.467435  -0.464025  -0.469809   
...                      ...        ...        ...        ...        ...   
2020-01-30 MSFT    -0.457010  -0.462958  -0.464188  -0.466514  -0.465248   
           ^GSPC   17.709853  17.879114  17.987061  18.004861  18.004861   
2020-01-31 AAPL    -0.901448  -0.901799  -0.904931  -0.905742  -0.905966   
           MSFT    -0.463403  -0.465725  -0.449934  -0.450002  -0.448696   
           ^GSPC   18.060043  18.039038  17.956716  17.948012  17.948012   

                       Volume  
Date       Ticker              
2016-01-04 AAPL           NaN  
           MSFT     -0.205048  
           ^GSPC    79.049091  
2016-01-05 AAPL     -0.987040  
           MSFT     -0.389154  
...                       ...  
2020-01-30 MSFT      0.628411  
           ^GSPC    72.399874  
2020-01-31 AAPL     -0.986825  
           MSFT     -0.275655  
           ^GSPC   124.276474  

[3081 rows x 6 columns]

In [84]:
aapl = data_pct.xs('AAPL', level='Ticker', drop_level=False)
display(aapl.Close.head(10))

Date        Ticker
2016-01-04  AAPL           NaN
2016-01-05  AAPL     -0.948968
2016-01-06  AAPL     -0.950067
2016-01-07  AAPL     -0.951539
2016-01-08  AAPL     -0.950100
2016-01-11  AAPL     -0.948736
2016-01-12  AAPL     -0.948037
2016-01-13  AAPL     -0.949765
2016-01-14  AAPL     -0.947352
2016-01-15  AAPL     -0.949460
Name: Close, dtype: float64

In [85]:
log_returns = np.log(close.dropna()).diff()
display(log_returns.head())

Ticker,AAPL,MSFT,^GSPC
Date,,,
2016-01-04,NaN,NaN,NaN
2016-01-05,-0.025379,0.004552,0.002010
2016-01-06,-0.019764,-0.018332,-0.013202
2016-01-07,-0.043121,-0.035402,-0.023986
2016-01-08,0.005274,0.003062,-0.010898


In [86]:
layout = gobjs.Layout(
    title='Result',
    yaxis=dict(
        title='Cumulative log returns',
        titlefont=dict(
            size=26,
            color='#7f7f7f'
        )
    )
)
    
axis = []

for d in log_returns:
    axi = gobjs.Scatter(
                x=log_returns.index,
                y=log_returns[d].cumsum(),
                name = d,
                opacity = 1)
    axis.append(axi)
    
fig = dict(data=axis, layout=layout)
plty.iplot(fig)


In [115]:


layout = gobjs.Layout(
    title='Result',
    yaxis=dict(
        title='Total relative returns (%)',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)
axis = []

for d in log_returns:
    axi = gobjs.Scatter(
                x=log_returns.index,
                y= 100 * (np.exp(log_returns[d].cumsum()) -1),
                name = d,
                opacity = 1)
    axis.append(axi)
    
fig = dict(data=axis, layout=layout)
plty.iplot(fig)

# Buy and Hold

Buying on the first day and selling on the last day



In [101]:
# transposing the prices of the last day (instead of a row vector it will be a column vector)

r_t = log_returns.tail(1).transpose()
display(r_t)

Date,2020-01-31
Ticker,
AAPL,-0.045352
MSFT,-0.014869
^GSPC,-0.017864


In [100]:
# setting the stock importance for my wallet, in this case each of the three stocks got the same importance 
# e.g. if i will invest $100 in this wallet, this amount will be equaly divided among the three stocks - $33.33 for each one

weights_vector = pd.DataFrame(1 / 3, index=r_t.index, columns=r_t.columns)
display(weights_vector)

Date,2020-01-31
Ticker,
AAPL,0.333333
MSFT,0.333333
^GSPC,0.333333


In [102]:
portfolio_log_return = weights_vector.transpose().dot(r_t)
portfolio_log_return

Date,2020-01-31
Date,
2020-01-31,-0.026028


In [103]:
weights_matrix = pd.DataFrame(1 / 3, index=log_returns.index, columns=log_returns.columns)
weights_matrix.head()

Ticker,AAPL,MSFT,^GSPC
Date,,,
2016-01-04,0.333333,0.333333,0.333333
2016-01-05,0.333333,0.333333,0.333333
2016-01-06,0.333333,0.333333,0.333333
2016-01-07,0.333333,0.333333,0.333333
2016-01-08,0.333333,0.333333,0.333333


In [108]:
temp_var = weights_matrix.dot(log_returns.transpose())
display(log_returns.transpose())
display(temp_var.tail())

Date,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-11,2016-01-12,2016-01-13,2016-01-14,2016-01-15,...,2020-01-17,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,NaN,-0.025379,-0.019764,-0.043121,0.005274,0.016063,0.014409,-0.026047,0.021635,-0.024308,...,0.011010,-0.006800,0.003563,0.004804,-0.002886,-0.029846,0.027897,0.020716,-0.001450,-0.045352
MSFT,NaN,0.004552,-0.018332,-0.035402,0.003062,-0.000573,0.009136,-0.021836,0.028069,-0.040736,...,0.005581,-0.003597,-0.004816,0.006137,-0.010128,-0.016865,0.019406,0.015472,0.027817,-0.014869
^GSPC,NaN,0.002010,-0.013202,-0.023986,-0.010898,0.000853,0.007773,-0.025282,0.016558,-0.021836,...,0.003855,-0.002656,0.000289,0.001140,-0.009083,-0.015856,0.010003,-0.000867,0.003129,-0.017864


Date,2016-01-04,2016-01-05,2016-01-06,2016-01-07,2016-01-08,2016-01-11,2016-01-12,2016-01-13,2016-01-14,2016-01-15,...,2020-01-17,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-27,NaN,-0.006272,-0.017099,-0.03417,-0.000854,0.005447,0.010439,-0.024388,0.022087,-0.02896,...,0.006815,-0.004351,-0.000321,0.004027,-0.007366,-0.020856,0.019102,0.011774,0.009832,-0.026028
2020-01-28,NaN,-0.006272,-0.017099,-0.03417,-0.000854,0.005447,0.010439,-0.024388,0.022087,-0.02896,...,0.006815,-0.004351,-0.000321,0.004027,-0.007366,-0.020856,0.019102,0.011774,0.009832,-0.026028
2020-01-29,NaN,-0.006272,-0.017099,-0.03417,-0.000854,0.005447,0.010439,-0.024388,0.022087,-0.02896,...,0.006815,-0.004351,-0.000321,0.004027,-0.007366,-0.020856,0.019102,0.011774,0.009832,-0.026028
2020-01-30,NaN,-0.006272,-0.017099,-0.03417,-0.000854,0.005447,0.010439,-0.024388,0.022087,-0.02896,...,0.006815,-0.004351,-0.000321,0.004027,-0.007366,-0.020856,0.019102,0.011774,0.009832,-0.026028
2020-01-31,NaN,-0.006272,-0.017099,-0.03417,-0.000854,0.005447,0.010439,-0.024388,0.022087,-0.02896,...,0.006815,-0.004351,-0.000321,0.004027,-0.007366,-0.020856,0.019102,0.011774,0.009832,-0.026028


In [109]:
portfolio_log_returns = pd.Series(np.diag(temp_var), index=log_returns.index)
portfolio_log_returns = portfolio_log_returns[1:]
display(portfolio_log_returns)

Date
2016-01-05   -0.006272
2016-01-06   -0.017099
2016-01-07   -0.034170
2016-01-08   -0.000854
2016-01-11    0.005447
                ...   
2020-01-27   -0.020856
2020-01-28    0.019102
2020-01-29    0.011774
2020-01-30    0.009832
2020-01-31   -0.026028
Length: 1026, dtype: float64

In [111]:


layout = gobjs.Layout(
    title='Result',
    yaxis=dict(
        title='Portfolio cumulative log returns',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)
    
axis = gobjs.Scatter(
            x=portfolio_log_returns.index,
            y=portfolio_log_returns.cumsum(),
            opacity = 1)
    
fig = dict(data=[axis], layout=layout)
plty.iplot(fig)



In [112]:
total_relative_returns = (np.exp(portfolio_log_returns.cumsum()) - 1)

In [114]:
layout = gobjs.Layout(
    title='Result',
    yaxis=dict(
        title='Portfolio total relative returns (%)',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)
    
axis = gobjs.Scatter(
            x = total_relative_returns.index,
            y = total_relative_returns * 100,
            opacity = 1)
    
fig = dict(data=[axis], layout=layout)
plty.iplot(fig)